In [3]:
#imports

import os
# This line changes tensorflow log level and suppresses warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.framework.ops import EagerTensor
from tensorflow.python.ops.resource_variable_ops import ResourceVariable
import time

E0000 00:00:1731728738.995212    2616 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731728739.019055    2616 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# Data Import
train_dataset = h5py.File('datasets/train_signs.h5', "r")
test_dataset = h5py.File('datasets/test_signs.h5', "r")

In [5]:
# Data Import Check
train_dataset['train_set_x']

<HDF5 dataset "train_set_x": shape (1080, 64, 64, 3), type "|u1">

In [6]:
# Data Preparation
x_train = tf.data.Dataset.from_tensor_slices(train_dataset['train_set_x'])
y_train = tf.data.Dataset.from_tensor_slices(train_dataset['train_set_y'])

x_test = tf.data.Dataset.from_tensor_slices(test_dataset['test_set_x'])
y_test = tf.data.Dataset.from_tensor_slices(test_dataset['test_set_y'])

I0000 00:00:1731728742.622211    2616 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6717 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:0a:00.0, compute capability: 6.1


In [7]:
# Data Type Check
type(x_train)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [8]:
# Train and Test Data spec check
print('x_train spec: ', x_train.element_spec)
print('x_test spec: ', x_test.element_spec)
print('y_train spec: ', y_train.element_spec)
print('y_test spec: ', y_test.element_spec)

x_train spec:  TensorSpec(shape=(64, 64, 3), dtype=tf.uint8, name=None)
x_test spec:  TensorSpec(shape=(64, 64, 3), dtype=tf.uint8, name=None)
y_train spec:  TensorSpec(shape=(), dtype=tf.int64, name=None)
y_test spec:  TensorSpec(shape=(), dtype=tf.int64, name=None)


In [9]:
# Verifying x_train is iterable or not
print(next(iter(x_train)))

tf.Tensor(
[[[227 220 214]
  [227 221 215]
  [227 222 215]
  ...
  [232 230 224]
  [231 229 222]
  [230 229 221]]

 [[227 221 214]
  [227 221 215]
  [228 221 215]
  ...
  [232 230 224]
  [231 229 222]
  [231 229 221]]

 [[227 221 214]
  [227 221 214]
  [227 221 215]
  ...
  [232 230 224]
  [231 229 223]
  [230 229 221]]

 ...

 [[119  81  51]
  [124  85  55]
  [127  87  58]
  ...
  [210 211 211]
  [211 212 210]
  [210 211 210]]

 [[119  79  51]
  [124  84  55]
  [126  85  56]
  ...
  [210 211 210]
  [210 211 210]
  [209 210 209]]

 [[119  81  51]
  [123  83  55]
  [122  82  54]
  ...
  [209 210 210]
  [209 210 209]
  [208 209 209]]], shape=(64, 64, 3), dtype=uint8)


In [10]:
# Normalize Image
def normalize(image):
    image = tf.cast(image, tf.float32) / 256.0
    image = tf.reshape(image, [-1, 1])
    return image

In [11]:
new_train = x_train.map(normalize)
new_test = x_test.map(normalize)

In [12]:
# Train and Test Data spec check
print('new_train spec: ', new_train.element_spec)
print('new_test spec: ', new_test.element_spec)

new_train spec:  TensorSpec(shape=(12288, 1), dtype=tf.float32, name=None)
new_test spec:  TensorSpec(shape=(12288, 1), dtype=tf.float32, name=None)


In [13]:
print(next(iter(new_train)))

tf.Tensor(
[[0.88671875]
 [0.859375  ]
 [0.8359375 ]
 ...
 [0.8125    ]
 [0.81640625]
 [0.81640625]], shape=(12288, 1), dtype=float32)


In [16]:
# Linear Function using Tensorflow
def linear_function():
    # Y = WX + b

    np.random.seed(1)

    X = tf.constant(np.random.randn(3, 1))
    W = tf.constant(np.random.randn(4, 3))
    b = tf.constant(np.random.randn(4, 1))
    Y = tf.add(tf.matmul(W, X), b)

    return Y

In [17]:
result = linear_function()
print('basic linear function result: ', result)

basic linear function result:  tf.Tensor(
[[-2.15657382]
 [ 2.95891446]
 [-1.08926781]
 [-0.84538042]], shape=(4, 1), dtype=float64)


In [18]:
# Sigmoid using tensorflow
def sigmoid(z):
    z = tf.cast(z, tf.float32)
    a = tf.keras.activations.sigmoid(z)
    return a

In [19]:
result = sigmoid(-1)
print ("type: " + str(type(result)))
print ("dtype: " + str(result.dtype))
print ("sigmoid(-1) = " + str(result))
print ("sigmoid(0) = " + str(sigmoid(0.0)))
print ("sigmoid(12) = " + str(sigmoid(12)))

type: <class 'tensorflow.python.framework.ops.EagerTensor'>
dtype: <dtype: 'float32'>
sigmoid(-1) = tf.Tensor(0.26894143, shape=(), dtype=float32)
sigmoid(0) = tf.Tensor(0.5, shape=(), dtype=float32)
sigmoid(12) = tf.Tensor(0.9999938, shape=(), dtype=float32)


In [20]:
# One hot matrix in tensorflow
def one_hot_matrix(label, depth=8):

    one_hot = tf.one_hot(label, depth, axis=0)
    one_hot = tf.reshape(one_hot, [depth, 1])
    
    return one_hot

In [22]:
label = tf.constant(1)
depth = 10
result = one_hot_matrix(label, depth)
print(result)

tf.Tensor(
[[0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(10, 1), dtype=float32)


In [23]:
# Using one hot matrix
new_y_test = y_test.map(one_hot_matrix)
new_y_train = y_train.map(one_hot_matrix)

In [27]:
print('new_y_test: ', next(iter(new_y_test)))
print('new_y_train: ', next(iter(new_y_train)))

new_y_test:  tf.Tensor(
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]], shape=(8, 1), dtype=float32)
new_y_train:  tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]], shape=(8, 1), dtype=float32)


In [28]:
# Parameters Initialization using tf
def initialize_parameters():
    initializer = tf.keras.initializers.GlorotNormal(seed=1)
    W1 = tf.Variable(initializer(shape=(25, 12288)))
    b1 = tf.Variable(initializer(shape=(25, 1)))
    W2 = tf.Variable(initializer(shape=(12, 25)))
    b2 = tf.Variable(initializer(shape=(12, 1)))
    W3 = tf.Variable(initializer(shape=(6, 12)))
    b3 = tf.Variable(initializer(shape=(6, 1)))

    params = {
        'W1': W1,
        'b1': b1,
        'W2': W2,
        'b2': b2,
        'W3': b3
    }
    return params

In [29]:
params = initialize_parameters()
print('parameters: ', params)

parameters:  {'W1': <tf.Variable 'Variable:0' shape=(25, 12288) dtype=float32, numpy=
array([[ 0.00183502, -0.00714326,  0.00914526, ..., -0.01221176,
         0.01656458,  0.00526992],
       [ 0.02264025,  0.0067227 ,  0.00795862, ...,  0.00284724,
         0.01910819,  0.00122853],
       [-0.00163333, -0.00861273, -0.01398893, ..., -0.00741132,
         0.02071251, -0.026417  ],
       ...,
       [-0.00126929,  0.01729332,  0.02082342, ...,  0.01709594,
         0.00429358, -0.00733263],
       [ 0.00275988,  0.00419503,  0.00949959, ...,  0.01241149,
        -0.0270652 ,  0.01501104],
       [-0.00145541,  0.02459595,  0.00339064, ..., -0.02478788,
         0.02716016, -0.00306428]], dtype=float32)>, 'b1': <tf.Variable 'Variable:0' shape=(25, 1) dtype=float32, numpy=
array([[ 0.03993344],
       [-0.15545043],
       [ 0.19901742],
       [-0.24872702],
       [-0.28688195],
       [-0.12585858],
       [-0.01201915],
       [ 0.14756493],
       [-0.00472601],
       [-0.4423628